# Loading of Dataset

In [1]:
!pip install pydub

In [2]:
!pip install ffmpeg

In [3]:
from os import path
from pydub import AudioSegment
mp3_audio = AudioSegment.from_mp3("D:\Sid\Tri_NIT_ML01\common_voice_ta_34938436.mp3")
mp3_audio.export("output.wav", format="wav")

C:\Users\siddh\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
C:\Users\siddh\anaconda3\lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [4]:
# import required modules
import subprocess
  
# convert mp3 to wav file
subprocess.call(['ffmpeg', '-i', 'common_voice_ta_34938436.mp3',
                 'converted_to_wav_file.wav'])

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [5]:
path.exists("D:\Sid\Tri_NIT_ML01\common_voice_ta_34938436.mp3")

True

In [6]:
!pip install torchaudio

In [7]:
import torchaudio

filename = "D:\Sid\Tri_NIT_ML01\common_voice_ta_34938436.mp3"
waveform, sample_rate = torchaudio.load(filename)

new_sample_rate = 16000

RuntimeError: Error opening 'D:\\Sid\\Tri_NIT_ML01\\common_voice_ta_34938436.mp3': File contains data in an unknown format.

In [ ]:
import torchaudio

In [ ]:
filename = "D:\Sid\Tri_NIT_ML01\common_voice_ta_34938436.mp3"
waveform, sample_rate = torchaudio.load(filename)

In [ ]:
!pip install soundfile

In [ ]:
filename = "D:\Sid\Tri_NIT_ML01\common_voice_ta_34938436.mp3"
waveform, sample_rate = torchaudio.load(filename)

In [ ]:
!pip install PySoundFile

In [ ]:
filename = "D:\Sid\Tri_NIT_ML01\common_voice_ta_34938436.mp3"
waveform, sample_rate = torchaudio.load(filename)

In [ ]:
!pip install sox

In [ ]:
!pip install librosa

In [8]:
import librosa
import numpy as np

# Load the audio file into a NumPy array
filename = "D:\Sid\Tri_NIT_ML01\common_voice_ta_34938436.wav"
audio_data, sample_rate = librosa.load(filename)

# Convert the audio data to a float32 NumPy array
audio_data = np.asarray(audio_data, dtype=np.float32)

C:\Users\siddh\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [9]:
audio_data

array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 2.0301112e-09,
       3.7992569e-09, 0.0000000e+00], dtype=float32)

In [10]:
import torch
from torch import nn

In [11]:
torch_tensor = torch.from_numpy(audio_data)

# Audio Preprocessing

In [12]:
def preprocess_audio(audio_array, sample_rate=44100, target_sr=16000):
    audio_array = librosa.resample(audio_array, sample_rate, target_sr)
    audio_array = audio_array / np.max(np.abs(audio_array))
    audio_array = librosa.effects.trim(audio_array, top_db=30, frame_length=512, hop_length=64)[0]
    audio_array = librosa.util.normalize(audio_array, norm=np.inf, axis=0)
    
    return audio_array

In [13]:
preprocess_audio(audio_data)

C:\Users\siddh\AppData\Local\Temp\ipykernel_18528\1660540950.py:2: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  audio_array = librosa.resample(audio_array, sample_rate, target_sr)


array([ 1.5554485e-04, -8.6654481e-06,  9.4710593e-04, ...,
        6.8765553e-06, -3.7375049e-04, -2.3233258e-04], dtype=float32)

In [14]:
def generate_spectrogram(audio_array, sample_rate=44100, n_fft=1024, hop_length=512):
    spectrogram = librosa.stft(audio_array, n_fft=n_fft, hop_length=hop_length)
    spectrogram = np.abs(spectrogram)
    spectrogram = torch.FloatTensor(spectrogram)
    return spectrogram

spectrogram = generate_spectrogram(audio_data)
spectrogram = spectrogram.unsqueeze(0)
spectrogram = spectrogram.permute(0, 2, 1)

spectrogram

tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [4.8437e-06, 2.0622e-06, 9.5338e-06,  ..., 3.5602e-09,
          1.7919e-09, 9.8768e-10],
         [4.3408e-06, 6.9780e-05, 1.3596e-04,  ..., 1.7516e-08,
          1.2543e-08, 6.4176e-09],
         ...,
         [2.6557e-04, 4.2780e-04, 4.2456e-04,  ..., 2.1753e-08,
          1.4093e-08, 7.9568e-09],
         [6.5349e-04, 2.0462e-04, 3.1281e-04,  ..., 6.8456e-09,
          1.0818e-08, 3.2371e-09],
         [8.1132e-04, 7.0383e-04, 5.8204e-04,  ..., 3.8921e-08,
          4.6556e-08, 5.4736e-08]]])

## Building a WaveGAN Model with Generator and a Discriminator Network

In [16]:
import torch.nn.functional as F

class Generator(nn.Module):
    def __init__(self, latent_dim=100, output_channels=1):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.output_channels = output_channels
        
        self.fc1 = nn.Linear(latent_dim, 1024)
        self.conv1 = nn.ConvTranspose1d(1024, 512, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.ConvTranspose1d(512, 256, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.ConvTranspose1d(256, 128, kernel_size=4, stride=2, padding=1)
        self.conv4 = nn.ConvTranspose1d(128, output_channels, kernel_size=4, stride=2, padding=1)
        
        self.bn1 = nn.BatchNorm1d(1024)
        self.bn2 = nn.BatchNorm1d(512)
        self.bn3 = nn.BatchNorm1d(256)
        self.bn4 = nn.BatchNorm1d(128)
    
    def forward(self, z):
        x = F.relu(self.bn1(self.fc1(z)))
        x = F.relu(self.bn2(self.conv1(x)))
        x = F.relu(self.bn3(self.conv2(x)))
        x = F.relu(self.bn4(self.conv3(x)))
        x = torch.tanh(self.conv4(x))
        return x

class Discriminator(nn.Module):
    def __init__(self, input_channels=1):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=25, stride=4, padding=11)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=25, stride=4, padding=11)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=25, stride=4, padding=11)
        self.conv4 = nn.Conv1d(256, 512, kernel_size=25, stride=4, padding=11)
        self.fc1 = nn.Linear(512 * 16, 1)
        
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)
        self.bn4 = nn.BatchNorm1d(512)
    
    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.conv1(x)), 0.2)
        x = F.leaky_relu(self.bn2(self.conv2(x)), 0.2)
        x = F.leaky_relu(self.bn3(self.conv3(x)), 0.2)
        x = F.leaky_relu(self.bn4(self.conv4(x)), 0.2)
        x = x.view(-1, 512 * 16)
        x = self.fc1(x)
        return x

    
class WaveGAN(nn.Module):
    def __init__(self, generator, discriminator):
        super(WaveGAN, self)._init_()
        self.generator = generator
        self.discriminator = discriminator
        
    def forward(self, z):
        x = self.generator(z)
        out = self.discriminator(x)
        return x, out

generator = Generator()
discriminator = Discriminator()
z = torch.from_numpy(audio_data)
x_fake = generator(z)
d_fake = discriminator(x_fake)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x88112 and 100x1024)

In [ ]:

import torch.nn.functional as F

class Generator(nn.Module):
    def __init__(self, input_size, num_filters, output_size):
        super(Generator, self).__init__()
        self.fc = nn.Linear(input_size, num_filters * 4 * 4)
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(num_filters * 4),
            nn.ConvTranspose2d(num_filters * 4, num_filters * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters * 2),
            nn.ConvTranspose2d(num_filters * 2, num_filters, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters),
            nn.ConvTranspose2d(num_filters, output_size, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        out = self.fc(z)
        out = out.view(-1, num_filters * 4, 4, 4)
        out = self.conv_blocks(out)
        return out
    
generator = Generator(input_size=1,num_filters=32,output_size=1)
z = spectrogram
x_fake = generator(z)
x_fake

In [ ]:
import torch.optim as optim


class Generator(nn.Module):
    def __init__(self, latent_dim=100, n_channels=1):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.n_channels = n_channels
        
        self.fc = nn.Sequential(
            nn.Linear(latent_dim, 128 * 16 * 8),
            nn.BatchNorm1d(128 * 16 * 8),
            nn.ReLU(True),
        )
        
        self.conv = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=(4, 8), stride=(2, 4), padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, n_channels, kernel_size=(4, 8), stride=(2, 4), padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        z = self.fc(z)
        z = z.view(z.size(0), 128, 16, 8)
        out = self.conv(z)
        return out

class Discriminator(nn.Module):
    def __init__(self, n_channels=1):
        super(Discriminator, self)._init_()
        self.n_channels = n_channels
        
        self.conv = nn.Sequential(
            nn.Conv2d(n_channels, 64, kernel_size=(4, 8), stride=(2, 4), padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=(4, 8), stride=(2, 4), padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
        )
        
        self.fc = nn.Sequential(
            nn.Linear(128 * 16 * 8, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        out = self.fc(x)
        return out

class WaveGAN(nn.Module):
    def __init__(self, generator, discriminator):
        super(WaveGAN, self)._init_()
        self.generator = generator
        self.discriminator = discriminator
        
    def forward(self, z):
        x = self.generator(z)
        out = self.discriminator(x)
        return x, out

def WaveGAN_loss(fake_out, real_out, fake_audio, real_audio):
    adversarial_loss = torch.mean(-torch.log(fake_out) - torch.log(1 - real_out))
    feature_matching_loss = torch.mean(torch.abs(torch.mean(fake_audio, dim=3) - torch.mean(real_audio, dim=3)))
    return adversarial_loss + 10 * feature_matching_loss


data = librosa.util.normalize(audio_data)
data = librosa.feature.melspectrogram(data, n_mels=128)


batch_size = 64
latent_dim = 100
n_epochs = 100
lr = 0.0002


wave = WaveGAN()

g = Generator()
d = Discriminator()
a = wave(g,d)

In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim=1, num_filters=100, kernel_size=4):
        super(Generator, self).__init__()
        self.z_dim = z_dim
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        
        self.fc = nn.Linear(z_dim, 16 * 16 * num_filters)
        self.bn = nn.BatchNorm2d(16 * 16 * num_filters)
        self.relu = nn.ReLU()
        self.conv_transpose = nn.ConvTranspose2d(num_filters, 1, kernel_size, stride=2, padding=1, output_padding=1)
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, self.num_filters, 16, 16)
        x = self.bn(x)
        x = self.relu(x)
        x = self.conv_transpose(x)
        x = self.tanh(x)
        return x

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, num_filters, kernel_size):
        super(Discriminator, self).__init__()
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        
        self.conv = nn.Conv1d(1, num_filters, kernel_size, stride=2, padding=1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(32 * num_filters, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = x.view(-1, 32 * self.num_filters)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

In [ ]:
import torch.optim as optim


data = audio_data


generator = Generator()
discriminator = Discriminator(num_filters=32, kernel_size=16)


criterion = nn.BCELoss()
gen_optimizer = optim.Adam(generator.parameters(), lr=0.0002)
dis_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)


num_epochs = 100
for epoch in range(num_epochs):
    for i, audio in enumerate(data):
        dis_optimizer.zero_grad()
        real_labels = torch.ones(audio.size(0), 1)
        fake_labels = torch.zeros(audio.size(0), 1)
        
        real_outputs = discriminator(audio)
        real_loss = criterion(real_outputs, real_labels)
        
        noise = torch.randn(audio.size(0), 100)
        fake_audio = generator(noise)
        fake_outputs = discriminator(fake_audio.detach())
        fake_loss = criterion(fake_outputs, fake_labels)
        
        dis_loss = real_loss + fake_loss
        dis_loss.backward()
        dis_optimizer.step()

        gen_optimizer.zero_grad()
        noise = torch.randn(audio.size(0), 100)
        fake_audio = generator(noise)
        fake_outputs = discriminator(fake_audio)
        gen_loss = criterion(fake_outputs, real_labels)
        gen_loss.backward()
        gen_optimizer.step()
        
        if i % 100 == 0:
            print("Epoch: {}, Iteration: {}, Dis Loss: {:.4f}, Gen Loss: {:.4f}".format(
                epoch, i, dis_loss.item(), gen_loss.item()))

noise = torch.randn(1, 100)
fake_audio = generator(noise)